In [0]:
%python
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import *

In [0]:
client_id = 'b53ca2d2-b8ee-4282-84db-112e9512b018'
tenant_id = 'd4a2d791-9631-4acd-8b35-d758e27620fe'
# removed client_secret in order to upload in github
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
          

In [0]:
try:
  dbutils.fs.mount(
  source = "abfss://demo@springboardde.dfs.core.windows.net/",
  mount_point = "/mnt/springboardde/demo",
  extra_configs = configs)
except Exception as e:
  pass
display(dbutils.fs.ls("/mnt/springboardde/demo"))

path,name,size,modificationTime
dbfs:/mnt/springboardde/demo/credit card.csv,credit card.csv,664848,1745694106000
dbfs:/mnt/springboardde/demo/loan.csv,loan.csv,43967,1745694106000
dbfs:/mnt/springboardde/demo/txn.csv,txn.csv,7885867,1745694107000


# LOAN DATASET #

In [0]:
#read in load dataset
loan_df = spark.read.options(delimiter=',').csv('/mnt/springboardde/demo/loan.csv',inferSchema=True,header=True)

In [0]:
# Print the schema of the loan dataset
loan_df.printSchema()

root
 |-- Customer_ID: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Family Size: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Expenditure: integer (nullable = true)
 |-- Use Frequency: integer (nullable = true)
 |-- Loan Category: string (nullable = true)
 |-- Loan Amount: string (nullable = true)
 |-- Overdue: integer (nullable = true)
 |--  Debt Record: string (nullable = true)
 |--  Returned Cheque: integer (nullable = true)
 |--  Dishonour of Bill: integer (nullable = true)



In [0]:
# Print the first 5 lines of the loan dataset
display(loan_df.head(5))

Customer_ID,Age,Gender,Occupation,Marital Status,Family Size,Income,Expenditure,Use Frequency,Loan Category,Loan Amount,Overdue,Debt Record,Returned Cheque,Dishonour of Bill
IB14001,30,MALE,BANK MANAGER,SINGLE,4,50000,22199,6,HOUSING,"10,00,000",5,"42,898",6,9
IB14008,44,MALE,PROFESSOR,MARRIED,6,51000,19999,4,SHOPPING,"50,000",3,"33,999",1,5
IB14012,30,FEMALE,DENTIST,SINGLE,3,58450,27675,5,TRAVELLING,"75,000",6,"20,876",3,1
IB14018,29,MALE,TEACHER,MARRIED,5,45767,12787,3,GOLD LOAN,"6,00,000",7,"11,000",0,4
IB14022,34,MALE,POLICE,SINGLE,4,43521,11999,3,AUTOMOBILE,"2,00,000",2,"43,898",1,2


In [0]:
# Print the number of columns in the loan dataset
print("Number of columns in Loan Dataset:", len(loan_df.columns))

Number of columns in Loan Dataset: 15


In [0]:
# Print the number of rows in the loan dataset
print("Number of rows in Loan Dataset:", loan_df.count())

Number of rows in Loan Dataset: 500


In [0]:
# Print the count of distinct records in the loan dataset
print("Count of distinct records in Loan Dataset:", loan_df.distinct().count())

Count of distinct records in Loan Dataset: 500


In [0]:
# find the number of loans in each category
results = loan_df.groupBy("Loan Category").agg(
    count("*").alias("count")
)
results.show()

+------------------+-----+
|     Loan Category|count|
+------------------+-----+
|           HOUSING|   67|
|        TRAVELLING|   53|
|       BOOK STORES|    7|
|       AGRICULTURE|   12|
|         GOLD LOAN|   77|
|  EDUCATIONAL LOAN|   20|
|        AUTOMOBILE|   60|
|          BUSINESS|   24|
|COMPUTER SOFTWARES|   35|
|           DINNING|   14|
|          SHOPPING|   35|
|       RESTAURANTS|   41|
|       ELECTRONICS|   14|
|          BUILDING|    7|
|        RESTAURANT|   20|
|   HOME APPLIANCES|   14|
+------------------+-----+



In [0]:
# find the number of people who have taken more than 1 lack loan
# Clean the loan amount column by removing spaces, commas, and quotes
loan_df = loan_df.withColumn(
    "Loan_Amount_Clean", 
    regexp_replace(regexp_replace(col("Loan Amount"), "[, ]", ""), '"', "").cast("double")
)

# Filter for loans greater than 100,000 (1 lakh)
loans_over_one_lakh = loan_df.filter(col("Loan_Amount_Clean") > 100000)

# Count the number of people with loans over 1 lakh
result = loans_over_one_lakh.count()

print(f"Number of people who have taken loans greater than 1 lakh: {result}")


Number of people who have taken loans greater than 1 lakh: 450


In [0]:
# find the number of people with income greater than 60000 rupees
high_income_people = loan_df.filter(col("Income") > 60000)

# Count the number of people with income > 60000
results = high_income_people.count()

print(f"Number of people with income greater than 60,000 rupees: {results}")

Number of people with income greater than 60,000 rupees: 198


In [0]:
# find the number of people with 2 or more returned cheques and income less than 50000
filtered_people = loan_df.filter(
    (col(" Returned Cheque") >= 2) & 
    (col("Income") < 50000)
)

# Count the number of people matching these criteria
results = filtered_people.count()

print(f"Number of people with 2+ returned cheques and income less than 50,000: {results}")


Number of people with 2+ returned cheques and income less than 50,000: 137


In [0]:
# find the number of people with 2 or more returned cheques and are single
filtered_people = loan_df.filter(
    (col(" Returned Cheque") >= 2) & 
    (col("Marital Status") == "SINGLE")
)

# Count the number of people matching these criteria
results = filtered_people.count()

print(f"Number of people with 2+ returned cheques and are single: {results}")


Number of people with 2+ returned cheques and are single: 111


In [0]:
# find the  number of people with expenditure over 50000 a month 
high_expenditure_people = loan_df.filter(col("Expenditure") > 50000)

# Count the number of people with expenditure > 50000
results = high_expenditure_people.count()

print(f"Number of people with expenditure over 50,000: {results}")


Number of people with expenditure over 50,000: 6


# CREDIT CARD DATASET #

In [0]:
# Load the credit card dataset
credit_df = spark.read.options(delimiter=',').csv('/mnt/springboardde/demo/credit card.csv',inferSchema=True,header=True)

In [0]:
# Print the schema of the credit card dataset
credit_df.printSchema()

root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



In [0]:
# Print the number of columns in the credit card dataset
print("Number of columns in Credit Card Dataset:", len(credit_df.columns))

Number of columns in Credit Card Dataset: 13


In [0]:
# Print the number of rows in the credit card dataset
print("Number of rows in Credit Card Dataset:", credit_df.count())

Number of rows in Credit Card Dataset: 10000


In [0]:
# Print the number of distinct records in the credit card dataset
print("Count of distinct records in Loan Dataset:", credit_df.distinct().count())

Count of distinct records in Loan Dataset: 10000


In [0]:
# Print the first 5 rows in the credit card dataset
credit_df.show(5)

+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|      0.0|            1|             1|      101348.88|     1|
|        2|  15647311|    Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        3|  15619304|    Onio|        502|   France|Female| 42|     8| 159660.8|            3|             0|      113931.57|     1|
|        4|  15701354|    Boni|        699|   France|Female| 39|     1|      0.0|            2|             0|       93826.63|     0|
|        5|  15737888|Mitchell|        850|    Spain|Female| 4

In [0]:
# Find the number of members who are elgible for credit card
eligible_people = credit_df.filter(
    (col("CreditScore") > 650) 
    & (col("Exited") == 0)
)

# Count the number of people matching these criteria
results = eligible_people.count()

print(f"Number of people who are elgible for credit card: {results}")


Number of people who are elgible for credit card: 4083


In [0]:
# Find the number of members who are  elgible and active in the bank
eligible_people = credit_df.filter(
    (col("CreditScore") > 650) 
    & (col("Exited") == 0)
    & (col("IsActiveMember") == 1)
)

# Count the number of people matching these criteria
results = eligible_people.count()

print(f"Number of people who are elgible and active: {results}")


Number of people who are elgible and active: 2312


In [0]:
# Find the credit card users in Spain 
results = credit_df.filter(col("Geography") == "Spain").count()
print(f"Number of people from Spain: {results}")

Number of people from Spain: 2477


In [0]:
# Find the credit card users with Estiamted Salary greater than 100000 and have exited the card
results = credit_df.filter(
    (col("EstimatedSalary") > 100000) 
    & (col("Exited") == 1)
)
results.select('CustomerId').show()

+----------+
|CustomerId|
+----------+
|  15634602|
|  15619304|
|  15574012|
|  15656148|
|  15699309|
|  15589475|
|  15738148|
|  15755196|
|  15602280|
|  15771573|
|  15702298|
|  15663706|
|  15762418|
|  15757535|
|  15804919|
|  15609618|
|  15661670|
|  15627360|
|  15713483|
|  15612350|
+----------+
only showing top 20 rows


In [0]:
# Find the credit card users with Estiamted Salary less than 100000 and have more than 1 products
credit_df = credit_df.withColumn("EstimatedSalary", col("EstimatedSalary").cast("double"))
credit_df = credit_df.withColumn("NumOfProducts", col("NumOfProducts").cast("integer"))

# Filter for users with Estimated Salary < 100000 AND NumOfProducts > 1
target_users = credit_df.filter(
    (col("EstimatedSalary") < 100000) & 
    (col("NumOfProducts") > 1)
)

# Show the matching users
target_users.select('CustomerId').show()


+----------+
|CustomerId|
+----------+
|  15701354|
|  15592531|
|  15792365|
|  15767821|
|  15737173|
|  15632264|
|  15600882|
|  15643966|
|  15788218|
|  15568982|
|  15725737|
|  15700772|
|  15656300|
|  15750181|
|  15659428|
|  15768193|
|  15789484|
|  15703793|
|  15779052|
|  15780961|
+----------+
only showing top 20 rows


# TRANSACTION DATASET #

In [0]:
# Load the transacton dataset
txn_df = spark.read.options(delimiter=',').csv('/mnt/springboardde/demo/txn.csv',inferSchema=True,header=True)

In [0]:
# Print the schema of the transacton dataset
txn_df.printSchema()

root
 |-- Account No: string (nullable = true)
 |-- TRANSACTION DETAILS: string (nullable = true)
 |-- VALUE DATE: string (nullable = true)
 |--  WITHDRAWAL AMT : double (nullable = true)
 |--  DEPOSIT AMT : double (nullable = true)
 |-- BALANCE AMT: double (nullable = true)



In [0]:
#COUNT OF TRANSACTION ON EVERY ACCOUNT
results = txn_df.groupBy("Account No").agg(
    count("*").alias("count")
)
results.show()


+-------------+-----+
|   Account No|count|
+-------------+-----+
|409000438611'| 4588|
|     1196711'|10536|
|     1196428'|48779|
|409000493210'| 6014|
|409000611074'| 1093|
|409000425051'|  802|
|409000405747'|   51|
|409000493201'| 1044|
|409000438620'|13454|
|409000362497'|29840|
+-------------+-----+



In [0]:
# Find the Maximum withdrawal amount for each account
results = txn_df.groupBy("Account No").agg(
    max(" WITHDRAWAL AMT ")
)
results.show()

+-------------+---------------------+
|   Account No|max( WITHDRAWAL AMT )|
+-------------+---------------------+
|409000438611'|                2.4E8|
|     1196711'|        4.594475464E8|
|     1196428'|                1.5E8|
|409000493210'|                1.5E7|
|409000611074'|             912000.0|
|409000425051'|               3.54E8|
|409000405747'|                1.7E8|
|409000493201'|            2500000.0|
|409000438620'|                4.0E8|
|409000362497'|        1.413662392E8|
+-------------+---------------------+



In [0]:
#MINIMUM WITHDRAWAL AMOUNT OF AN ACCOUNT
results = txn_df.groupBy("Account No").agg(
    min(" WITHDRAWAL AMT ")
)
results.show()

+-------------+---------------------+
|   Account No|min( WITHDRAWAL AMT )|
+-------------+---------------------+
|409000438611'|                  0.2|
|     1196711'|                 0.25|
|     1196428'|                 0.25|
|409000493210'|                 0.01|
|409000611074'|                120.0|
|409000425051'|                 1.25|
|409000405747'|                 21.0|
|409000493201'|                  2.1|
|409000438620'|                 0.34|
|409000362497'|                 0.97|
+-------------+---------------------+



In [0]:
#MAXIMUM DEPOSIT AMOUNT OF AN ACCOUNT
results = txn_df.groupBy("Account No").agg(
    max(" DEPOSIT AMT ")
)
results.show()

+-------------+------------------+
|   Account No|max( DEPOSIT AMT )|
+-------------+------------------+
|409000438611'|          1.7025E8|
|     1196711'|             5.0E8|
|     1196428'|     2.119594422E8|
|409000493210'|             1.5E7|
|409000611074'|         3000000.0|
|409000425051'|             1.5E7|
|409000405747'|           2.021E8|
|409000493201'|         1000000.0|
|409000438620'|           5.448E8|
|409000362497'|             2.0E8|
+-------------+------------------+



In [0]:
#MINIMUM DEPOSIT AMOUNT OF AN ACCOUNT
results = txn_df.groupBy("Account No").agg(
    min(" DEPOSIT AMT ")
)
results.show()

+-------------+------------------+
|   Account No|min( DEPOSIT AMT )|
+-------------+------------------+
|409000438611'|              0.03|
|     1196711'|              1.01|
|     1196428'|               1.0|
|409000493210'|              0.01|
|409000611074'|            1320.0|
|409000425051'|               1.0|
|409000405747'|             500.0|
|409000493201'|               0.9|
|409000438620'|              0.07|
|409000362497'|              0.03|
+-------------+------------------+



In [0]:
#sum of balance in every bank account
results = txn_df.groupBy("Account No").agg(
    sum("BALANCE AMT")
)
results.show()

+-------------+--------------------+
|   Account No|    sum(BALANCE AMT)|
+-------------+--------------------+
|409000438611'|-2.49486577068339...|
|     1196711'|-1.60476498101275E13|
|     1196428'| -8.1418498130721E13|
|409000493210'|-3.27584952132095...|
|409000611074'|       1.615533622E9|
|409000425051'|-3.77211841164998...|
|409000405747'|-2.43108047067000...|
|409000493201'|1.0420831829499985E9|
|409000438620'|-7.12291867951358...|
|409000362497'| -5.2860004792808E13|
+-------------+--------------------+



In [0]:
#Number of transaction on each date
results = txn_df.groupBy("VALUE DATE").agg(
    count("*").alias("count")
)
results.show()

+----------+-----+
|VALUE DATE|count|
+----------+-----+
| 23-Dec-16|  143|
|  7-Feb-19|   98|
| 21-Jul-15|   80|
|  9-Sep-15|   91|
| 17-Jan-15|   16|
| 18-Nov-17|   53|
| 21-Feb-18|   77|
| 20-Mar-18|   71|
| 19-Apr-18|   71|
| 21-Jun-16|   97|
| 17-Oct-17|  101|
|  3-Jan-18|   70|
|  8-Jun-18|  223|
| 15-Dec-18|   62|
|  8-Aug-16|   97|
| 17-Dec-16|   74|
|  3-Sep-15|   83|
| 21-Jan-16|   76|
|  4-May-18|   92|
|  7-Sep-17|   94|
+----------+-----+
only showing top 20 rows


In [0]:
#List of customers with withdrawal amount more than 1 lakh
txn_df = txn_df.withColumn(
    "WITH_Amount_Clean", 
    regexp_replace(regexp_replace(col(" WITHDRAWAL AMT "), "[, ]", ""), '"', "").cast("double")
)

# Filter for amounts greater than 100,000 (1 lakh)
results = txn_df.filter(col("WITH_Amount_Clean") > 100000)

results.select('Account No').show()

+-------------+
|   Account No|
+-------------+
|409000611074'|
|409000611074'|
|409000611074'|
|409000611074'|
|409000611074'|
|409000611074'|
|409000611074'|
|409000611074'|
|409000611074'|
|409000611074'|
|409000611074'|
|409000611074'|
|409000611074'|
|409000611074'|
|409000611074'|
|409000611074'|
|409000611074'|
|409000611074'|
|409000611074'|
|409000611074'|
+-------------+
only showing top 20 rows


In [0]:
dbutils.fs.unmount('/mnt/springboardde/demo')

/mnt/springboardde/demo has been unmounted.


True